# Data preprocessing

In [ ]:
import pandas as pd

In [ ]:
annual_gold = pd.read_csv('annual_gold.csv') # Annual gold prices
geopolitical_risk = pd.read_excel('geopolitical_risk.xlsx') # Escalation of events such as wars, terrorism and political tensions
gdp = pd.read_csv('gross_domestic_product.csv', skiprows=4) # Gross domestic product
exchange_rate = pd.read_csv('exchange_rate.csv', skiprows=4) # Relative price of one currency expressed in terms of US Dollar
inflation_rate = pd.read_csv('inflation_rate.csv', skiprows=4) # Rate of increase in prices over a given period of time

In [ ]:
relevant_countries = ['Canada', 'China', 'Euro area', 'Japan', 'United Kingdom', 'United States']

In [ ]:
# Renaming columns for convenience
annual_gold.rename(columns={'Date': 'year', 'Price': 'gold'}, inplace=True)

annual_gold.reset_index(drop=True, inplace=True)

# Taking data between 1999 and 2023
annual_gold = annual_gold[annual_gold['year'] >= 1999]

In [ ]:
# Exchange data for CAD, RMB, EUR, JPY, GBP, USD
exchange_data = exchange_rate[exchange_rate['Country Name'].isin(relevant_countries)].copy()

exchange_data.set_index('Country Name', inplace=True)

# Removing unused columns and transposing so years become rows
exchange_data.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 68'], axis=1, inplace=True)
exchange_data = exchange_data.T

# Renaming columns for convenience
exchange_data.columns.name = None
exchange_data.reset_index(inplace=True)
exchange_data.rename(columns={'Canada': 'CAD_exchange', 'China': 'RMB_exchange', 'Euro area': 'EUR_exchange', 'Japan': 'JPY_exchange',
                              'United Kingdom': 'GBP_exchange', 'United States': 'USD_exchange', 'index': 'year'}, inplace=True)

# Taking data between 1999 and 2023
exchange_data['year'] = exchange_data['year'].astype('int64')
exchange_data = exchange_data[exchange_data['year'] >= 1999]

In [ ]:
# Relevant inflation data for relevant countries
inflation_data = inflation_rate[inflation_rate['Country Name'].isin(relevant_countries)].copy()

inflation_data.set_index('Country Name', inplace=True)

# Removing unused columns and transposing so years become rows
inflation_data.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 68'], axis=1, inplace=True)
inflation_data = inflation_data.T

# Renaming columns for convinience
inflation_data.columns.name = None
inflation_data.reset_index(inplace=True)
inflation_data.rename(columns={'Canada': 'CAD_inflation', 'China': 'RMB_inflation', 'Euro area': 'EUR_inflation', 'Japan': 'JPY_inflation',
                               'United Kingdom': 'GBP_inflation', 'United States': 'USD_inflation', 'index': 'year'}, inplace=True)

# Extracting data between 1999 and 2023
inflation_data['year'] = inflation_data['year'].astype('int64')
inflation_data = inflation_data[inflation_data['year'] >= 1999]

In [ ]:
# GDP data for CAD, RMB, EUR, JPY, GBP, USD
gdp_data = gdp[gdp['Country Name'].isin(relevant_countries)].copy()
gdp_data.set_index('Country Name', inplace=True)

# Removing unused columns and transposing so years become rows
gdp_data.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 68'], axis=1, inplace=True)
gdp_data = gdp_data.T

# Renaming columns for convenience
gdp_data.columns.name = None
gdp_data.reset_index(inplace=True)
gdp_data.rename(columns={'Canada': 'CAD_gdp', 'China': 'RMB_gdp', 'Euro area': 'EUR_gdp', 'Japan': 'JPY_gdp',
                              'United Kingdom': 'GBP_gdp', 'United States': 'USD_gdp', 'index': 'year'}, inplace=True)

# Taking data between 1999 and 2023
gdp_data['year'] = gdp_data['year'].astype('int64')
gdp_data = gdp_data[gdp_data['year'] >= 1999]

In [ ]:
# Filter by date
geopolitical_risk['month'] = pd.to_datetime(geopolitical_risk['month'])
gpr_filtered = geopolitical_risk[(geopolitical_risk['month'] >= '1999-01-01') & (geopolitical_risk['month'] <= '2023-12-01')]

# Selecting countries based on codes
countries = {
    'USA': 'GPRHC_USA',
    'UK': 'GPRHC_GBR',
    'Canada': 'GPRHC_CAN',
    'Japan': 'GPRHC_JPN',
    'China': 'GPRHC_CHN'
}

# Valid Euro Area countries (other Euro Area countries are missing from dataset)
valid_euro_countries = {
    'Belgium': 'GPRHC_BEL', 'Finland': 'GPRHC_FIN', 'France': 'GPRHC_FRA',
    'Germany': 'GPRHC_DEU', 'Italy': 'GPRHC_ITA', 'Netherlands': 'GPRHC_NLD',
    'Portugal': 'GPRHC_PRT', 'Spain': 'GPRHC_ESP'
}

# Select and copy relevant columns
selected_columns = ['month'] + list(countries.values()) + list(valid_euro_countries.values())
gpr_selected = gpr_filtered[selected_columns].copy()

# Compute Euro Area average
gpr_selected['Euro Area'] = gpr_selected[list(valid_euro_countries.values())].mean(axis=1)

# Rename columns to human-readable names
for country, code in countries.items():
    gpr_selected[country] = gpr_selected[code]

# Prepare final monthly data
df_final = gpr_selected[['month', 'Canada', 'China', 'Euro Area', 'Japan', 'UK', 'USA']].copy()

# Create a new column for year
df_final['year'] = df_final['month'].dt.year

# Group by year and calculate the mean for each country
geopolitical_risk = df_final.groupby('year')[['Canada', 'China', 'Euro Area', 'Japan', 'UK', 'USA']].mean().reset_index()

# Rename columns to currency codes
geopolitical_risk.rename(columns={'Canada': 'CAD_risk', 'China': 'RMB_risk', 'Euro Area': 'EUR_risk',
                                  'Japan': 'JPY_risk', 'UK': 'GBR_risk', 'USA': 'USD_risk'}, inplace=True)

In [ ]:
X = annual_gold.merge(gdp_data, on='year') \
               .merge(exchange_data, on='year') \
               .merge(inflation_data, on='year') \
               .merge(geopolitical_risk, on='year')

# X.drop(columns=['year'], inplace=True)
X.tail(3)

,year,gold,CAD_gdp,RMB_gdp,EUR_gdp,GBP_gdp,JPY_gdp,USD_gdp,CAD_exchange,RMB_exchange,...,EUR_inflation,GBP_inflation,JPY_inflation,USD_inflation,CAD_risk,RMB_risk,EUR_risk,JPY_risk,GBR_risk,USD_risk
22,2021,1799.629,2.007472e+12,1.782046e+13,1.491774e+13,3.143323e+12,5.034621e+12,2.368117e+13,1.253877,6.448975,...,2.486504,2.518371,-0.233353,4.697859,0.098135,0.562014,0.144666,0.140980,0.436932,2.107411
23,2022,1800.602,2.161483e+12,1.788178e+13,1.445205e+13,3.114042e+12,4.256411e+12,2.600689e+13,1.301555,6.737158,...,8.471176,7.922049,2.497703,8.002800,0.300739,1.119953,0.408424,0.417404,1.082489,4.045435
24,2023,1942.666,2.142471e+12,1.779478e+13,1.578069e+13,3.380855e+12,4.204495e+12,2.772071e+13,1.349909,7.083998,...,5.784316,6.793967,3.268134,4.116338,0.220683,0.994963,0.266116,0.264220,0.818269,3.571373


In [ ]:
X = X[24:]

# Downloading .csv file

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 24 to 24
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gold           1 non-null      float64
 1   CAD_gdp        1 non-null      float64
 2   RMB_gdp        1 non-null      float64
 3   EUR_gdp        1 non-null      float64
 4   GBP_gdp        1 non-null      float64
 5   JPY_gdp        1 non-null      float64
 6   USD_gdp        1 non-null      float64
 7   CAD_exchange   1 non-null      float64
 8   RMB_exchange   1 non-null      float64
 9   EUR_exchange   1 non-null      float64
 10  GBP_exchange   1 non-null      float64
 11  JPY_exchange   1 non-null      float64
 12  USD_exchange   1 non-null      float64
 13  CAD_inflation  1 non-null      float64
 14  RMB_inflation  1 non-null      float64
 15  EUR_inflation  1 non-null      float64
 16  GBP_inflation  1 non-null      float64
 17  JPY_inflation  1 non-null      float64
 18  USD_inflatio

In [ ]:
X

,gold,CAD_gdp,RMB_gdp,EUR_gdp,GBP_gdp,JPY_gdp,USD_gdp,CAD_exchange,RMB_exchange,EUR_exchange,...,EUR_inflation,GBP_inflation,JPY_inflation,USD_inflation,CAD_risk,RMB_risk,EUR_risk,JPY_risk,GBR_risk,USD_risk
24,1942.666,2.142471e+12,1.779478e+13,1.578069e+13,3.380855e+12,4.204495e+12,2.772071e+13,1.349909,7.083998,0.92484,...,5.784316,6.793967,3.268134,4.116338,0.220683,0.994963,0.266116,0.26422,0.818269,3.571373


In [ ]:
X.to_csv('test_data.csv', index=False)